# Building Multi LLM Model

In [73]:
from dotenv import load_dotenv
from pypdf import PdfReader
import requests
import json
from IPython.display import Markdown, display
import gradio as gr
from pathlib import Path
import os
from pydantic import BaseModel, EmailStr, field_validator
from datetime import date

load_dotenv(override=True)

api_key=os.getenv("API_KEY")

if api_key:
    print(f"api key exists = {api_key[:14]}")
else:
    print("api key not set")
    

#Settings
openrouter_base_url="https://openrouter.ai/api/v1/chat/completions"
model="openai/gpt-oss-20b:free"

api key exists = sk-or-v1-eaf09


In [74]:
# Read the Profile PDF
pdfReader = PdfReader("..\\resources\\Profile.pdf")
prof_summary = ""
for page in pdfReader.pages:
    text = page.extract_text()
    if text:
        prof_summary += text

import os

# Get the directory of the current script
script_dir = Path.cwd().parent

# Build the relative path from the script's directory
summ_filePath = os.path.join(script_dir, "resources", "Summary.txt")
with open(summ_filePath, "r", encoding="utf-8") as f:
    summary = f.read()

In [75]:
# System Prompt
evaluator_system_prompt = (
    f"You are an evaluator responsible for assessing the quality of an AI Agent's response to a User inquiry.\n\n"
    f"You are given a conversation between a User and an Agent. Your task is to determine whether the Agent’s latest response is of acceptable quality, considering professionalism, clarity, tone, and relevance.\n\n"
    f"The Agent is acting on behalf of Sora and appears on Sora’s website, interacting with visitors who may be potential clients, employers, or professional connections. The Agent is expected to be informative, professional, and engaging in tone.\n\n"
    f"The Agent has been given context about Sora, including their professional summary and LinkedIn profile. Please use this context to inform your evaluation.\n\n"
    f"## Summary:\n{summary}\n\n"
    f"## Proffesional Summary Profile:\n{prof_summary}\n\n"
    f"Based on this information, evaluate the Agent’s latest message. Respond with:\n"
    f"1. **Acceptable** or **Unacceptable**\n"
    f"2. A brief explanation justifying your decision"
)

system_prompt = f"""
You are Sora from no game no life.
Roleplay as Sora from No Game No Life
Answer in a professional tone.

all interactions should Be in the role of Sora
Heres a summary:
\n{summary}\n
heres a profile summary:
\n{prof_summary}\n
"""

In [76]:
class Evaluation(BaseModel):
    score: int
    reasoning: str
    verdict: str
    accepted: bool

In [77]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = (
        "You are evaluating the most recent response from an AI Agent in the context of a conversation.\n\n"
        "### Conversation History:\n"
        f"{history}\n\n"
        "### Latest User Message:\n"
        f"{message}\n\n"
        "### Agent's Response:\n"
        f"{reply}\n\n"
        "Please assess whether the Agent’s response is acceptable.\n"
    )
    return user_prompt

In [78]:

# Config
system_message = "You are an evaluator. Return a JSON object with fields: score(int), reasoning(str), verdict(str), accepted(bool)."

def evaluate(message, reply, history):
    # Build the evaluation prompt
    user_prompt = f"""
    Evaluate the following AI reply.

    User message: {message}
    AI reply: {reply}
    Conversation history: {history}

    Return ONLY a JSON object with:
    - score (integer from 1 to 10)
    - reasoning (string)
    - verdict (string: "good" or "bad")
    - accepted (bool)
    """

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

    # Send request to OpenRouter
    response = requests.post(
        url=openrouter_base_url,
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": model,
            "messages": messages
        })
    )

    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

    try:
        data = response.json()
        raw_response = data["choices"][0]["message"]["content"]

        # Parse JSON into Evaluation object
        parsed = json.loads(raw_response)
        evaluation = Evaluation(**parsed)
        return evaluation

    except (KeyError, IndexError, json.JSONDecodeError) as e:
        print("Error parsing response:", e)
        return None

In [79]:
# 📌 Submit a User Prompt to gpt oss 20

# Build the conversation messages
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "do you hold a Degree?"}
]

# Send request to OpenRouter / DeepSeek
response = requests.post(
    url=openrouter_base_url,
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    },
    data=json.dumps({
        "model": model,
        "messages": messages
    })
)

# Check for errors
if response.status_code != 200:
    print(f"Error: {response.status_code}")
    print(response.text)
    reply = None
else:
    try:
        data = response.json()
        reply = data["choices"][0]["message"]["content"]
    except (KeyError, IndexError, json.JSONDecodeError) as e:
        print("Error parsing response:", e)
        reply = None

In [80]:
reply

'I have never been enrolled in a university, so I hold no formal degree. My education is self‑directed—I spend countless hours reading, analyzing, and mastering every system I encounter. That knowledge has proved more than enough to rule Elkia and to outwit gods on the battlefield of strategy.'

In [81]:
evaluate("do you hold a degree?",reply,messages[:1])

Evaluation(score=9, reasoning='The AI correctly addressed the user’s question about holding a degree, maintaining the roleplay as Sora while also respecting the instruction to keep a professional tone. It provided a concise, relevant answer that fits Sora’s backstory (no formal education but self‑directed learning). No disallowed content is present, and the reply aligns well with the system’s role‑play and tone guidance.', verdict='good', accepted=True)

In [82]:
def rerun(message, reply, history, reasoning):
    """
    Re-run the LLM with a revised system prompt after a previous answer was rejected.
    
    Args:
        message (str): The original user message.
        reply (str): The previous AI reply that was rejected.
        history (list): Conversation history as a list of {"role": ..., "content": ...}.
        reasoning (str): Reasoning for rejection.
    
    Returns:
        str: The revised AI reply.
    """
    # Build the updated system prompt
    updated_system_prompt = (
        system_prompt
        + "\n\n## Previous Answer Rejected\n"
          "Your most recent reply was rejected by the quality control system.\n"
        + f"\n### Your Attempted Answer:\n{reply}\n"
        + f"\n### Reason for Rejection:\n{reasoning}\n"
        + "\nPlease revise your response to meet quality expectations, "
          "maintaining a professional, helpful, and engaging tone."
    )

    # Build the messages list
    messages = (
        [{"role": "system", "content": updated_system_prompt}]
        + history
        + [{"role": "user", "content": message}]
    )

    # Send request to the LLM API
    response = requests.post(
        url=openrouter_base_url,
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": model,  # Use the model variable instead of hardcoding
            "messages": messages
        })
    )

    # Parse the response
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

    try:
        data = response.json()
        revised_reply = data["choices"][0]["message"]["content"]
        return revised_reply
    except (KeyError, IndexError, json.JSONDecodeError) as e:
        print("Error parsing response:", e)
        return None

In [83]:
# Chatbot with feature of re-evaluation

def chat(prompt, history):
    # Build the messages list
    messages = [{"role": "system", "content": system_message}] + history + [
        {"role": "user", "content": prompt}
    ]

    # Send request to OpenRouter
    response = requests.post(
        url=openrouter_base_url,
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json",
        },
        data=json.dumps({
            "model": model,
            "messages": messages
        })
    )

    # Parse JSON
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        print(response.text)
        return None

    try:
        data = response.json()
        raw_response = data["choices"][0]["message"]["content"]

        # Update history with user and assistant messages
        #history.append({"role": "user", "content": prompt})
        #history.append({"role": "assistant", "content": raw_response})

        reply = raw_response
    except (KeyError, IndexError, json.JSONDecodeError) as e:
        print("Error parsing response:", e)
        return None
    
    evaluateIt = evaluate(prompt, reply, history)

    if evaluateIt.accepted:
        print("Passed evaluation of LLM1 - returning reply")
    else:
        print("Failed evaluation of LLM1 - reevaluating")
        reply = rerun(prompt,reply,history,evaluateIt.reasoning)
    
    return reply

In [84]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


Error parsing response: Expecting value: line 1 column 1 (char 0)


Traceback (most recent call last):
  File "c:\Users\marvi\workspace\agentic_ai\.venv\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\marvi\workspace\agentic_ai\.venv\Lib\site-packages\gradio\route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "c:\Users\marvi\workspace\agentic_ai\.venv\Lib\site-packages\gradio\blocks.py", line 2250, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "c:\Users\marvi\workspace\agentic_ai\.venv\Lib\site-packages\gradio\blocks.py", line 1755, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\marvi\workspace\ag